In [4]:
import scanpy
import pandas as pd
import anndata
import os
import hdf5plugin
import os
import pickle as pkl

import pandas as pd

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
import pybiomart
import mygene
from sklearn.preprocessing import MinMaxScaler
import numpy as np



In [5]:
path = "/home/hb/python/lof/deg/norman/norman_for_deg_processing_gsym.h5ad"
adata = anndata.read_h5ad(path)

/home/hb/anaconda3/envs/lof/lib/python3.11/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
obs = adata.obs
exp = adata.to_df().copy().T
exp

cell_barcode,AAACCTGAGAAACCAT-2,AAACCTGAGAAACCGC-6,AAACCTGAGAAAGTGG-8,AAACCTGAGAAGAAGC-1,AAACCTGAGAAGGTTT-7,AAACCTGAGACATAAC-5,AAACCTGAGACCACGA-2,AAACCTGAGACGCACA-3,AAACCTGAGAGATGAG-6,AAACCTGAGAGCTGCA-6,...,TTTGTCATCTCTAAGG-2,TTTGTCATCTGAGGGA-3,TTTGTCATCTGATTCT-1,TTTGTCATCTGCGGCA-2,TTTGTCATCTGGCGAC-8,TTTGTCATCTTATCTG-5,TTTGTCATCTTGCATT-7,TTTGTCATCTTGTACT-5,TTTGTCATCTTGTCAT-6,TTTGTCATCTTTAGTC-6
gene_name,,,,,,,,,,,,,,,,,,,,,
TSPAN6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DPM1,1.0,0.0,2.0,1.0,2.0,0.0,4.0,4.0,1.0,1.0,...,1.0,3.0,3.0,0.0,1.0,2.0,1.0,4.0,1.0,1.0
SCYL3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
FIRRM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
FGR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC122526780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRNCR1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LINC02635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
scaler = MinMaxScaler()
exp_values = scaler.fit_transform(exp.values)
exp_norm = pd.DataFrame(exp_values, columns=exp.columns, index=exp.index)
exp_norm

cell_barcode,AAACCTGAGAAACCAT-2,AAACCTGAGAAACCGC-6,AAACCTGAGAAAGTGG-8,AAACCTGAGAAGAAGC-1,AAACCTGAGAAGGTTT-7,AAACCTGAGACATAAC-5,AAACCTGAGACCACGA-2,AAACCTGAGACGCACA-3,AAACCTGAGAGATGAG-6,AAACCTGAGAGCTGCA-6,...,TTTGTCATCTCTAAGG-2,TTTGTCATCTGAGGGA-3,TTTGTCATCTGATTCT-1,TTTGTCATCTGCGGCA-2,TTTGTCATCTGGCGAC-8,TTTGTCATCTTATCTG-5,TTTGTCATCTTGCATT-7,TTTGTCATCTTGTACT-5,TTTGTCATCTTGTCAT-6,TTTGTCATCTTTAGTC-6
gene_name,,,,,,,,,,,,,,,,,,,,,
TSPAN6,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DPM1,0.005128,0.0,0.00722,0.003876,0.007722,0.0,0.01278,0.007782,0.004386,0.004348,...,0.003846,0.005894,0.013825,0.0,0.003205,0.004762,0.002591,0.006993,0.001965,0.001422
SCYL3,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.002381,0.000000,0.000000,0.000000,0.000000
FIRRM,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.003846,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.001422
FGR,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC122526780,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PRNCR1,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LINC02635,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
exp_norm = exp_norm.loc[~exp_norm.index.duplicated(keep='first')]
exp_norm

cell_barcode,AAACCTGAGAAACCAT-2,AAACCTGAGAAACCGC-6,AAACCTGAGAAAGTGG-8,AAACCTGAGAAGAAGC-1,AAACCTGAGAAGGTTT-7,AAACCTGAGACATAAC-5,AAACCTGAGACCACGA-2,AAACCTGAGACGCACA-3,AAACCTGAGAGATGAG-6,AAACCTGAGAGCTGCA-6,...,TTTGTCATCTCTAAGG-2,TTTGTCATCTGAGGGA-3,TTTGTCATCTGATTCT-1,TTTGTCATCTGCGGCA-2,TTTGTCATCTGGCGAC-8,TTTGTCATCTTATCTG-5,TTTGTCATCTTGCATT-7,TTTGTCATCTTGTACT-5,TTTGTCATCTTGTCAT-6,TTTGTCATCTTTAGTC-6
gene_name,,,,,,,,,,,,,,,,,,,,,
TSPAN6,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DPM1,0.005128,0.0,0.00722,0.003876,0.007722,0.0,0.01278,0.007782,0.004386,0.004348,...,0.003846,0.005894,0.013825,0.0,0.003205,0.004762,0.002591,0.006993,0.001965,0.001422
SCYL3,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.002381,0.000000,0.000000,0.000000,0.000000
FIRRM,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.003846,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.001422
FGR,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC122526780,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PRNCR1,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LINC02635,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
import pandas as pd
result = pd.read_csv("/home/hb/python/lof/perturbseq/gene_exp_for_viper/output/AHR.csv", index_col=0)

In [10]:
can_viper = obs[obs['condition'].isin(result.index)]
exp_extract = exp_norm.loc[:, can_viper.index]

In [16]:
obs[obs['condition']=='control']

,guide_identity,read_count,UMI_count,coverage,gemgroup,good_coverage,number_of_cells,condition
cell_barcode,,,,,,,,
AAACCTGAGAAGAAGC-1,NegCtrl0_NegCtrl0__NegCtrl0_NegCtrl0,1252,67,18.686567,1,True,2,control
AAACCTGAGCCCAACC-6,NegCtrl0_NegCtrl0__NegCtrl0_NegCtrl0,1156,51,22.666667,6,True,1,control
AAACCTGAGCGTCTAT-4,NegCtrl1_NegCtrl0__NegCtrl1_NegCtrl0,1388,95,14.610526,4,True,1,control
AAACCTGAGGCACATG-7,NegCtrl1_NegCtrl0__NegCtrl1_NegCtrl0,1289,70,18.414286,7,True,1,control
AAACCTGAGGGCTTGA-6,NegCtrl11_NegCtrl0__NegCtrl11_NegCtrl0,500,29,17.241379,6,True,1,control
...,...,...,...,...,...,...,...,...
TTTGTCATCAAGATCC-6,NegCtrl11_NegCtrl0__NegCtrl11_NegCtrl0,262,13,20.153846,6,True,1,control
TTTGTCATCAGTACGT-6,NegCtrl1_NegCtrl0__NegCtrl1_NegCtrl0,126,8,15.750000,6,True,1,control
TTTGTCATCGCCGTGA-7,NegCtrl1_NegCtrl0__NegCtrl1_NegCtrl0,1030,56,18.392857,7,True,1,control


In [88]:
for condition in can_viper.condition.unique():
    print(condition)
    _condition = can_viper[can_viper['condition']==condition].index.tolist() 
    _condition_exp = exp_extract.loc[:, _condition]
    _condition_exp.to_csv(f'/home/hb/python/lof/perturbseq/gene_exp_for_viper/input/{condition}.csv')

CEBPB
SNAI1
MAP2K6
CBL
CEBPE
RUNX1T1
S1PR2
KLF1
TSC22D1
SET
FOXA1
BAK1
ETS2
SPI1
MAP2K3
LYL1
ISL2
FOXO4
BCL2L11
MAML2
FOXF1
MAPK1
PRDM1
IKZF3
ZNF318
MAP4K3
FOXA3
DLX2
HOXB9
DUSP9
LHX1
HOXC13
BCORL1
MEIS1
HOXA13
FOXL2
TGFBR2
FEV
CDKN1B
SGK1
CKS1B
CSRNP1
MAP4K5
CEBPA
AHR
HES7
TP73
RREB1
TBX3
TBX2
CDKN1C
ZBTB25
JUN
FOSB
CBFA2T3
POU3F2
ZBTB10
CDKN1A
IRF1
PTPN1
ARID1A
ZBTB1
HNF4A
EGR1
CITED1
ZC3HAV1


In [24]:
_control = obs[obs['condition']=='control']
_control_exp = exp_norm.loc[:, _control.index]
_control_exp
_control_exp.to_csv(f'/home/hb/python/lof/perturbseq/gene_exp_for_viper/input/control.csv')

In [91]:
len(os.listdir("/home/hb/python/lof/perturbseq/gene_exp_for_viper/input"))

66

In [90]:
pd.read_csv("/home/hb/python/lof/perturbseq/gene_exp_for_viper/output/AHR.csv")

,Unnamed: 0,AAACGGGCAAGAGGCT.2,AAACGGGGTGCGAAAC.4,AAACGGGTCTAACTGG.3,AAAGATGAGCCGATTT.4,AAAGATGAGGTTCCTA.1,AAAGATGTCTGGTATG.5,AAAGCAACACATGGGA.5,AAAGCAATCAGGCAAG.8,AAATGCCGTATATGAG.3,...,TTTACTGGTGTTCTTT.5,TTTATGCCAATGGACG.6,TTTATGCCATCTACGA.1,TTTATGCGTAAGTGGC.5,TTTATGCTCATCGCTC.7,TTTATGCTCTCGAGTA.6,TTTCCTCCAGACGCCT.8,TTTCCTCCATTCCTGC.2,TTTGCGCAGGGTGTGT.7,TTTGGTTGTCGGCACT.7
0,NR2E3,-0.517694,-0.758608,-0.723561,-0.623943,-0.660007,-0.762766,-0.652599,-0.308834,-0.968606,...,-0.595964,-0.621837,-0.850651,-0.607069,-0.307886,-0.694604,-0.033478,-0.576989,-0.479598,-0.714512
1,ZBTB33,-0.495170,-0.192388,0.281264,-0.880827,-0.987543,-0.230623,-1.383842,-0.319670,-0.577029,...,-0.116085,0.117212,-0.064307,-0.765312,-0.803195,-0.384445,-0.660446,-0.696941,-0.625117,-0.017653
2,ZGLP1,-0.821020,-0.003222,-1.158345,-1.147689,-1.855852,-0.736468,-1.492600,-0.504162,-1.114389,...,-0.764423,-0.984040,-0.722725,-0.956508,-0.384392,-0.762302,-1.207244,-0.935965,-1.083420,-0.565178
3,HSFX1,-0.004585,-0.012444,0.022032,-0.007181,-0.015557,-0.009662,0.028487,-0.006483,-0.011151,...,-0.012284,-0.009898,-0.012757,-0.008446,0.035965,-0.011976,-0.012395,-0.005336,0.029829,0.041433
4,NR1H3,-0.022315,-0.939733,-0.892417,-1.082997,-1.429624,-1.729507,-0.756891,-0.238062,-1.265137,...,-0.925747,-1.014767,-0.679350,-0.465598,-0.796087,-1.020680,-0.419609,-0.370129,-0.750276,-0.510877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,MED13,-1.188249,-0.051180,-0.491849,-1.474556,-0.479834,-0.660839,0.385813,-1.208548,0.360460,...,0.731866,0.269414,0.584077,-1.153845,0.308395,-0.321530,0.293191,0.267614,-1.353959,0.374940
5754,CDC42,2.184103,2.968275,2.576222,3.685547,3.531421,2.525335,4.128747,2.206673,2.533640,...,3.501619,2.594157,2.980504,3.095397,2.723287,2.572742,2.275748,2.765595,2.591467,2.040531
5755,FGFBP1,-0.029460,-0.035155,-0.035371,-0.026414,-0.042956,-0.041282,0.001793,-0.025249,-0.033414,...,-0.025534,0.017786,-0.040273,0.033972,-0.033499,-0.001958,-0.029681,0.020914,-0.029597,-0.030394
5756,PPP4R1,0.153061,-0.533275,0.165311,0.615513,-0.003789,0.099474,-0.111589,0.100579,0.101859,...,0.943630,0.019894,-0.040606,0.037463,0.474051,0.237618,0.686013,0.544660,0.165815,0.344878
